In [ ]:
"use strict";

In [ ]:
const asTable   = require ('as-table');
const log   = require ('ololog').noLocate;
const ansi  = require ('ansicolor').nice;
const ccxt  = require ('ccxt');
const interval = require('interval-promise');
var { Series, DataFrame } = require('pandas-js');
const sqlite3 = require('sqlite3').verbose();
var util = require('util');
var nj = require('numjs');
const { google } = require('googleapis');
var CircularBuffer = require("circular-buffer");

In [ ]:
let INTERVAL = 60
let TABLE = "Bitfinex_900_15"
let QUOTE_ASSET = "BTC"
let ASSET_NUM = 15
let HISTORIC_SIZE = 90
let FEATURE_NUM = 3
let VOLUME_AVERAGE_DAYS = 30
let PROJECT='axiom-209217'
let MODEL='Bitfinex_900_15'
let VERSION='bitfinex_btc_900_6'

In [ ]:
let db = new sqlite3.Database('./database/Live.db', (err) => {
  if (err) {
    console.error(err.message);
  }
  console.log('Connected to the database.');
});

In [ ]:
var buf = new CircularBuffer(HISTORIC_SIZE);

In [ ]:
let create_sql = util.format('CREATE TABLE IF NOT EXISTS  %s (date INTEGER, baseAsset varchar(20), quoteAsset varchar(20), high FLOAT, low FLOAT,  close FLOAT, volume FLOAT,  PRIMARY KEY (date, baseAsset));', TABLE)        

In [ ]:
let insert_sql = util.format('INSERT INTO %s VALUES (?,?,?,?,?,?,?)', TABLE);

In [ ]:
let top_sql = util.format('SELECT baseAsset, SUM(volume) AS total_volume FROM %s WHERE date>=? GROUP BY baseAsset ORDER BY total_volume DESC LIMIT ?;', TABLE)  

In [ ]:
db.run(create_sql);

In [ ]:
const exchange  = new ccxt.bitfinex ({ verbose: false});  

In [18]:
//bitfinex_trader.json
let key = 'KHYMo1wzIoU3uDcYn5qcO8lr3wHPVz5r3mcSFjr43XT';
let secret = "xQ0WERcmGjdLWAwPM4QV4jYyUbNzvwAleiawLQKlZJx"

In [19]:
const BFX = require('bitfinex-api-node')
const bfxRest = new BFX({
  apiKey: key,
  apiSecret: secret,

  ws: {
    autoReconnect: true,
    seqAudit: true,
    packetWDDelay: 10 * 1000
  }
}).rest
bfxRest.get_symbols((err, res) => {
	if (err) console.log(err)
	console.log(result)
})


TypeError: bfxRest.get_symbols is not a function

Error: constructor takes an object since version 2.0.0, see:
https://github.com/bitfinexcom/bitfinex-api-node#version-200-breaking-changes


In [ ]:
var prev_w = [[0.06666666666666667,
  0.06666666666666667,
  0.06666666666666667,
  0.06666666666666667,
  0.06666666666666667,
  0.06666666666666667,
  0.06666666666666667,
  0.06666666666666667,
  0.06666666666666667,
  0.06666666666666667,
  0.06666666666666667,
  0.06666666666666667,
  0.06666666666666667,
  0.06666666666666667,
  0.06666666666666667]];

In [ ]:
// RISKLESS ASSET SHOULD BE CHANGED TO USD

(async function() {


        interval(async () => {
            var top_assets =[];
            var top_assets_obj = {};
            var inp_row = nj.zeros([FEATURE_NUM,ASSET_NUM]).tolist();
            
            var d = new Date(); 
            d.setDate(d.getDate() - VOLUME_AVERAGE_DAYS);
            var earliest = d.getTime();            

            db.all(top_sql, [earliest, ASSET_NUM],(err, rows ) => {
              if (err) {
                console.error(err);
              }
                 if (rows.length!=ASSET_NUM){                     
                         console.error("the sqlite error happend")
                 } else{
                          rows.forEach((row, i) => {                
                                // creates a list of the most volumnous assets
                                // reset top assets to this value
                                top_assets[i] = row['baseAsset']
                                top_assets_obj[row['baseAsset']] = i
                          });
                }
            });   
            
            // PREVIOUS VECTOR

            // TODO FETCH CANDLES INSTEAD
            var tickers = await exchange.fetchTickers ();                         
            
            // GET DIFFERENCE BETWEEN PREVIOUS VECTOR AND CURRENT VECTOR           
            await Object.values(tickers).map(async (tick) => { 
                var pair = tick['symbol'].split('/')
                if (pair[1]==QUOTE_ASSET) {
                      db.run(insert_sql, [
                          Math.round(tick['timestamp']),
                          pair[0],
                          pair[1],
                          tick['high'],
                          tick['low'],
                          tick['close'],
                          tick['baseVolume']
                      ]);              
                      if (top_assets.includes(pair[0])) {         
                          // get the index to which the asset resolves
                          // insert a new value into each respective 
                          // column of the inp vector                            
                          var asset_index = top_assets_obj[pair[0]];
                          
                          //console.log(pair[0]+":"+asset_index.toString());
                          
                          // close
                          inp_row[0][asset_index]= tick['close'];
                          
                          // high
                          inp_row[1][asset_index]= tick['high'];
                          
                          // low
                          inp_row[2][asset_index] = tick['low'];                 
                      }                               
                }                 
            });
            
            var row = nj.array(inp_row).reshape(3,15, 1)
            
            console.log('---------------------------------------------------------------------');
            console.log(row);
            console.log('---------------------------------------------------------------------');
            
            //console.log(row);
            buf.enq(row);            
            if(buf.size()==HISTORIC_SIZE) {                
                var inp = nj.stack(buf.toarray(), -1).tolist();

                console.log(inp);

                console.log("=============================================================================");

                // Send request to google cloud machine learning engine 
                google.auth.getApplicationDefault( (err, authClient, projectId) => {
                    if (err) {
                        console.log('Authentication failed because of ', err);
                        res.status(401).send('Authentication failed');

                    } else {

                        var ml = google.ml({
                            version: 'v1',
                            auth: authClient
                        });

                       var instances ={'instances': [{
                           'input': inp,
                           'previous_w':prev_w,
                       }]};

                        ml.projects.predict({
                            name: util.format('projects/%s/models/%s/versions/%s', PROJECT, MODEL, VERSION),
                            resource: instances
                        }, 
                        function(err, result) {
                            if (err) {
                                return callback(err);
                            }
                            
                            // FLATTEN: CLOSE ALL OPEN ORDERS AND POSITIONS

                            var payload = {
                               request: '/v1/order/cancel/all',
                               nonce: Date.now().toString(),
                            }

                            const BFX = require('bitfinex-api-node')
                            const bfxRest = new BFX(apiKey, apiSecretKey, {version: 1}).rest

                            bfxRest.cancel_all_orders((err, res) => {
                              if (err) console.log(err)
                              console.log(result)
                            })                       
                            
                            // Get all Active positions
//                             [{
//                               "id":943715,
//                               "symbol":"btcusd",
//                               "status":"ACTIVE",
//                               "base":"246.94",
//                               "amount":"1.0",
//                               "timestamp":"1444141857.0",
//                               "swap":"0.0",
//                               "pl":"-2.22042"
//                             }]
                            
                            var payload = {
                               request: '/v1/positions',
                               nonce: Date.now().toString(),
                            }

                            // or use bitfinex-api-node
                            const BFX = require('bitfinex-api-node')
                            const bfxRest = new BFX(apiKey, apiSecretKey, {version: 1}).rest

                            bfxRest.active_positions((err, res) => {
                                 if (err) console.log(err)
                                console.log(result)

                                       // place orders to zero out position  in usd
                                await Object.items(top_assets).map(async (tick) => { 

                                    // view how to authenticate here:
                                    // https://docs.bitfinex.com/v1/docs/rest-auth

                                    var payload = {
                                       request: '/v1/positions/claim',
                                       nonce: Date.now().toString(),
                                       id: 943715,
                                       amount: '1.0',
                                    }

                                    // or use bitfinex-api-node

                                    const BFX = require('bitfinex-api-node')
                                    const bfxRest = new BFX(apiKey, apiSecretKey, {version: 1}).rest

                                    bfxRest.claim_position(943715, '1.0', (err, res) => {
                                      if (err) console.log(err)
                                      console.log(result)
                                    })  
                                }
                            })
                            
                                
                            // GET CURRENT BALANCE 
                            // FOR EACH SCALAR VALUE IN THE OUTPUT PV CREATE A MARKET ORDER
                                
                                
                                
                            // cancel all orders
                            
                            
                            // place orders to match 
                            // the portfolio value
                            
                            
                            
                            
                        });

                   }
                });


                console.log("=============================================================================");
                
             }
            

            
        }, (1000*INTERVAL), {})    
})();

In [ ]:
   for(var asset of assets) {
                 for(var feature of feature_list) {
                      let sql;
                      if (feature == "close") {
                            sql = util.format("SELECT date+300 AS date_norm, close FROM {table} WHERE"
                               " date_norm>={start}"
                               " and date_norm%{period}=0 and coin=\"{coin}\"", TABLE);

                      } else if (feature == "open"){
                            sql = util.format("SELECT date+{period} AS date_norm, open FROM {table} WHERE"
                               " date_norm>={start}"
                               " and date_norm%{period}=0 and coin=\"{coin}\"", TABLE);


                      } else if (feature == "volume"){
                            sql = util.format("SELECT date_norm, SUM(volume)"+
                               " FROM (SELECT date+{period}-(date%{period}) "
                               "AS date_norm, volume, coin FROM {table})"
                               " WHERE date_norm>={start} and coin=\"{coin}\""
                               " GROUP BY date_norm", TABLE);

                      } else if (feature == "high"){
                            sql = util.format("SELECT date_norm, MAX(high)" +
                               " FROM (SELECT date+{period}-(date%{period})"
                               " AS date_norm, high, coin FROM {table})"
                               " WHERE date_norm>={start} and coin=\"{coin}\""
                               " GROUP BY date_norm", TABLE);

                      } else if (feature == "low"){
                            sql = util.format("SELECT date_norm, MIN(low)" +
                                " FROM (SELECT date+{period}-(date%{period})"
                                " AS date_norm, low, coin FROM {table})"
                                " WHERE date_norm>={start} and coin=\"{coin}\""
                                " GROUP BY date_norm", TABLE);

                      }
                     db.all(sql, [],(err, rows ) => {
                          if (err) {
                            throw err;
                          }
                           console.log("------------------------------------------------------------------------------------------------------------------------------------------------------");
                           rows.forEach((row) => {                
                              console.log(row);                  
                           });
                           console.log("------------------------------------------------------------------------------------------------------------------------------------------------------");
                    });
                 }
            }
 // get features for each coin in the 
            if feature == "close":
                sql = (.format(
                           table=self.__table, start=start, period=period, coin=coin))
            elif feature == "open":
                sql = (.format(
                           table=self.__table,start=start, period=period, coin=coin))
            elif feature == "volume":
                sql = (.format(
                           table=self.__table,period=period,start=start,coin=coin))
            elif feature == "high":
                sql = (.format(
                           table=self.__table,period=period,start=start,coin=coin))
            elif feature == "low":
                sql = (.format(
                            table=self.__table,period=period,start=start,coin=coin))
            

In [ ]:
inp = nj

In [ ]:
var nj = require('numjs');
const { google } = require('googleapis');
let PROJECT='axiom-209217'
let MODEL='Bitfinex_900_15'
let VERSION='bitfinex_btc_900_6'
// Send request to google cloud machine learning engine 
google.auth.getApplicationDefault( (err, authClient, projectId) => {
    if (err) {
        console.log('Authentication failed because of ', err);
        res.status(401).send('Authentication failed');

    } else {

        var ml = google.ml({
            version: 'v1',
            auth: authClient
        });
        
       inp = nj.random([3,15,90]).tolist();
       prev_w = nj.random([1,15]).tolist();

       var instances ={'instances': [{
           'input': inp,
           'previous_w':prev_w,
       }]};
        
        ml.projects.predict({
            name: util.format('projects/%s/models/%s/versions/%s', PROJECT, MODEL, VERSION),
            resource: instances
        }, 
        function(err, result) {
            if (err) {
                return callback(err);
            }
            console.log(JSON.stringify(result.data));

            // Derive the new portfolio vector value

        });

   }
});